Fuente: https://github.com/ugik/notebooks/blob/master/Tensorflow%20chat-bot%20model.ipynb
        https://github.com/ugik/notebooks/blob/master/Tensorflow%20chat-bot%20response.ipynb
        https://www.youtube.com/watch?v=1lwddP0KUEg&t=1763s

In [ ]:
import  nltk
from    nltk.stem import WordNetLemmatizer
from    tensorflow.keras.models import load_model
import  json
import  numpy as np
import  tensorflow as tf
import  random
import  pickle

lemmatizer = WordNetLemmatizer()
ERROR_THRESHOLD = 0.25


In [ ]:
model =     load_model('./data/chatbot_model.h5')
words =     pickle.load(open('./data/words.pkl', 'rb'))
classes =   pickle.load(open('./data/classes.pkl', 'rb'))
intents =   json.loads(open('./data/intents.json', encoding = "utf-8").read())

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    return(np.array(bag))
    
def classify(sentence):
    print('\n USUARIO:', sentence)
    bow =   bag_of_words(sentence)
    res =   model.predict(np.array([bow]))[0]
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability':str(r[1])})
    return return_list

def get_response(intents_list, intent_json): 
    tag = intents_list[0]['intent']
    list_of_intents = intents['intents']
    for i in list_of_intents: 
        if i['tag'] == tag: 
            result = random.choice(i['responses'])
            break
    return result


In [ ]:
while True: 
    message = input('')
    ints = classify(message)
    #print('ints', ints)
    result = get_response(ints, intents)
    print('Bot:', result)
    